In [1]:
import os

os.chdir(r'/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/')

In [2]:
import buffalo.ingestion as ingestion

In [3]:
ingestion.configuration.Configuration.print()

{'storage_folder': '../__cached_data', 'api_keys': {<API.ADVANTAGE: 1>: '2rrNROO0beX90lPH7ixQOp0mT_9SwF0d'}, 'watched_tickers': ['IBM', 'MSFT']}


In [4]:
advantage_endpoint = ingestion.advantage.AdvantageStockGrepper()

In [8]:
advantage_endpoint.stock_download(
    symbol='AAPL',
    interval='daily',
    year_slice=None, ## Intraday 
    adjusted=True
)

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2023-02-24,147.11,147.19,145.720,146.71,146.710000,55385542,0.0,1.0
1,2023-02-23,150.09,150.34,147.240,149.40,149.400000,48394249,0.0,1.0
2,2023-02-22,148.87,149.95,147.160,148.91,148.910000,51011305,0.0,1.0
3,2023-02-21,150.20,151.30,148.405,148.48,148.480000,58867230,0.0,1.0
4,2023-02-17,152.35,153.00,150.850,152.55,152.550000,59144118,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5862,1999-11-05,84.62,88.37,84.000,88.31,0.670165,3721500,0.0,1.0
5863,1999-11-04,82.06,85.37,80.620,83.62,0.634574,3384700,0.0,1.0
5864,1999-11-03,81.62,83.25,81.000,81.50,0.618485,2932700,0.0,1.0
5865,1999-11-02,78.00,81.69,77.310,80.25,0.608999,3564600,0.0,1.0


In [10]:
advantage_endpoint.listing_info_download(date='2022-02-24')

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,AXS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AAC,Ares Acquisition Corporation - Class A,NYSE,Stock,2021-03-25,NaN,Active
...,...,...,...,...,...,...,...
12043,ZYNE,Zynerba Pharmaceuticals Inc,NASDAQ,Stock,2015-08-05,NaN,Active
12044,ZYRX,Global Earnings Capital Ltd,NASDAQ,Stock,2007-07-13,NaN,Active
12045,ZYXI,Zynex Inc,NASDAQ,Stock,2012-07-23,NaN,Active
12046,ZZK,NaN,NYSE ARCA,Stock,2020-07-22,NaN,Active


In [11]:
advantage_endpoint.trend_indicator_download(
    function='SMA',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

,time,SMA
0,2023-02-23 19:22,130.718
1,2023-02-23 19:21,130.757
2,2023-02-23 18:59,130.767
3,2023-02-23 16:52,130.793
4,2023-02-23 16:09,130.815
...,...,...
3687,2023-02-10 09:32,133.688
3688,2023-02-10 09:31,133.711
3689,2023-02-10 09:30,133.683
3690,2023-02-10 09:08,133.664


In [12]:
advantage_endpoint.trend_indicator_download(
    function='T3',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

,time,T3
0,2023-02-23 19:22,130.7402
1,2023-02-23 19:21,130.7634
2,2023-02-23 18:59,130.7821
3,2023-02-23 16:52,130.7983
4,2023-02-23 16:09,130.8049
...,...,...
3642,2023-02-10 10:17,134.2102
3643,2023-02-10 10:16,134.1742
3644,2023-02-10 10:15,134.1395
3645,2023-02-10 10:14,134.1051


In [16]:
advantage_endpoint.cycle_indicator_download(
    function='ULTOSC',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

,time,ULTOSC
0,2023-02-23 19:22,35.1847
1,2023-02-23 19:21,46.7668
2,2023-02-23 18:59,20.1347
3,2023-02-23 16:52,28.3456
4,2023-02-23 16:09,45.9584
...,...,...
3668,2023-02-10 09:51,50.9523
3669,2023-02-10 09:50,54.6262
3670,2023-02-10 09:49,56.8546
3671,2023-02-10 09:48,49.9106


In [13]:
advantage_endpoint.market_news_sentiment_download('IBM', sort='Latest', limit=50)[0]

,index,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,overall_sentiment_score,overall_sentiment_label,items,sentiment_score_definition,relevance_score_definition,topic,topic_relevance_score
0,0,"More than 110,000 tech-sector employees have l...",https://www.marketwatch.com/story/more-than-11...,20230224T172900,[James Rogers],Layoffs.fyi's tally of 2023 global tech layoff...,https://images.mktw.net/im-662749/social,MarketWatch,Top Stories,www.marketwatch.com,-0.113191,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",Technology,0.5
1,0,"More than 110,000 tech-sector employees have l...",https://www.marketwatch.com/story/more-than-11...,20230224T172900,[James Rogers],Layoffs.fyi's tally of 2023 global tech layoff...,https://images.mktw.net/im-662749/social,MarketWatch,Top Stories,www.marketwatch.com,-0.113191,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",Manufacturing,0.5
2,1,'Ukraine is open for business': Western compan...,https://www.marketwatch.com/story/ukraine-is-o...,20230224T131800,[James Rogers],"Ukraine's GDP fell by 30.4% in 2022, according...",https://images.mktw.net/im-608875/social,MarketWatch,Top Stories,www.marketwatch.com,-0.038608,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",Life Sciences,0.166667
3,1,'Ukraine is open for business': Western compan...,https://www.marketwatch.com/story/ukraine-is-o...,20230224T131800,[James Rogers],"Ukraine's GDP fell by 30.4% in 2022, according...",https://images.mktw.net/im-608875/social,MarketWatch,Top Stories,www.marketwatch.com,-0.038608,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",Energy & Transportation,0.166667
4,1,'Ukraine is open for business': Western compan...,https://www.marketwatch.com/story/ukraine-is-o...,20230224T131800,[James Rogers],"Ukraine's GDP fell by 30.4% in 2022, according...",https://images.mktw.net/im-608875/social,MarketWatch,Top Stories,www.marketwatch.com,-0.038608,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",Technology,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,49,These five-star stock funds can help you play ...,https://www.marketwatch.com/story/these-five-s...,20230202T121100,[Philip van Doorn],Sylvia Jablonksi of Defiance ETFs describes tw...,https://images.mktw.net/im-713940/social,MarketWatch,Top Stories,www.marketwatch.com,0.097526,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",Manufacturing,0.25
201,49,These five-star stock funds can help you play ...,https://www.marketwatch.com/story/these-five-s...,20230202T121100,[Philip van Doorn],Sylvia Jablonksi of Defiance ETFs describes tw...,https://images.mktw.net/im-713940/social,MarketWatch,Top Stories,www.marketwatch.com,0.097526,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",Earnings,0.796627
202,49,These five-star stock funds can help you play ...,https://www.marketwatch.com/story/these-five-s...,20230202T121100,[Philip van Doorn],Sylvia Jablonksi of Defiance ETFs describes tw...,https://images.mktw.net/im-713940/social,MarketWatch,Top Stories,www.marketwatch.com,0.097526,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",Real Estate & Construction,0.25
203,49,These five-star stock funds can help you play ...,https://www.marketwatch.com/story/these-five-s...,20230202T121100,[Philip van Doorn],Sylvia Jablonksi of Defiance ETFs describes tw...,https://images.mktw.net/im-713940/social,MarketWatch,Top Stories,www.marketwatch.com,0.097526,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15:

In [15]:
advantage_endpoint.market_news_sentiment_download('IBM', sort='relevance', limit=50)[1]

,index,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,overall_sentiment_score,overall_sentiment_label,items,sentiment_score_definition,relevance_score_definition,ticker,ticker_relevance_score,ticker_sentiment_score,ticker_sentiment_label
0,0,IBM posts biggest annual sales increase in mor...,https://www.marketwatch.com/story/ibm-posts-bi...,20230125T215200,"[, Levi Sumagaysay, Jeremy C. Owens]",International Business Machines Corp. increase...,https://images.mktw.net/im-639870/social,MarketWatch,Top Stories,www.marketwatch.com,0.093375,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",IBM,0.983605,0.167656,Somewhat-Bullish
1,1,Tiger Global-Backed Quantum Computing Startup ...,https://www.benzinga.com/news/22/11/29742968/t...,20221116T102148,[Anusuya Lahiri],"Algorithmiq, a Helsinki-based quantum computin...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,0.186770,Somewhat-Bullish,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",IBM,0.980738,0.372322,Bullish
2,2,IBM Introduces 433 Qubit 'Osprey' Processor At...,https://www.benzinga.com/news/22/11/29652548/i...,20221110T103927,[Anusuya Lahiri],International Business Machines Corp IBM kicke...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,0.068142,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",APG,0.113597,-0.11637,Neutral
3,2,IBM Introduces 433 Qubit 'Osprey' Processor At...,https://www.benzinga.com/news/22/11/29652548/i...,20221110T103927,[Anusuya Lahiri],International Business Machines Corp IBM kicke...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,0.068142,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",VOD,0.224903,0.0,Neutral
4,2,IBM Introduces 433 Qubit 'Osprey' Processor At...,https://www.benzinga.com/news/22/11/29652548/i...,20221110T103927,[Anusuya Lahiri],International Business Machines Corp IBM kicke...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,0.068142,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",IBM,0.977729,0.068482,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,49,Global 2000 List 2022: The Top 200,https://www.forbes.com/sites/forbesstaff/2022/...,20220512T101500,[Isabel Contreras],"War, pandemic and sluggish markets hit the wor...",https://imageio.forbes.com/specials-images/ima...,Forbes,Business,www.forbes.com,0.010745,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",PEP,0.009638,0.005483,Neutral
406,49,Global 2000 List 2022: The Top 200,https://www.forbes.com/sites/forbesstaff/2022/...,20220512T101500,[Isabel Contreras],"War, pandemic and sluggish markets hit the wor...",https://imageio.forbes.com/specials-images/ima...,Forbes,Business,www.forbes.com,0.010745,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",CCZ,0.009638,0.012229,Neutral
407,49,Global 2000 List 2022: The Top 200,https://www.forbes.com/sites/forbesstaff/2022/...,20220512T101500,[Isabel Contreras],"War, pandemic and sluggish markets hit the wor...",https://imageio.forbes.com/specials-images/ima...,Forbes,Business,www.forbes.com,0.010745,Neutral,50,x <= -0.35: Bearish; -0.35 < x <= -0.15: Somew...,"0 < x <= 1, with a higher score indicating hig...",IDCBY,0.009638,0.000569,Neutral
408,49,Global 2000 List 2022: The Top 200,https://www.forbes.com/sites/forbesstaff/2022/...,20220512T101500,[Isabel Contreras],"War, pandemic and sluggish markets hit the wor...",https://imageio.forbes.com/specials-images/ima...,Forbes,Business,www.for

In [ ]:
import sqlite3


fp = r'~/newsqlitefile.sqlite'

import pandas as pd

a = pd.DataFrame({"a": range(10), "v": range(1, 11)})

newconn, _ = sqlite3.connect(fp)
a.to_sql("a", newconn)

newconn.close()